In [1]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: export-month-cbgs-logit.ipynb
#
# BY: Dmitry Sedov 
#
# CREATED: Mon Mar 16 2020
#
# DESC: This code produces tables with CBG-level destination CBG choices.
#
# EXEC:
#      
################################################################################
################################################################################

In [2]:
############################### Libraries ######################################

import sqlalchemy as db
import pandas as pd
import os
import numpy as np

################################################################################

In [3]:
############################ Constants and settings ############################

# Pandas display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

year = 2018
month = 10
output_folder_path = '/home/user/projects/urban/data/output/spatial-demand/two-stage-model' 

################################################################################

In [4]:
######################### PostgreSQL connection ################################

engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
connection = engine.connect()

################################################################################

In [5]:
################# CBG-to-CBG dataset creation SQL statements ###################

# Create a restaurants table with visits breakdown by home CBG
cbgs_visits_breakdown_table_statement = """
CREATE TEMPORARY TABLE cbgs_visits_long AS (
    SELECT 
        c.census_block_group AS destination_cbg,
        c.raw_visit_count,
        c.raw_visitor_count,
        (json_each(c.visitor_home_cbgs)).*
    FROM
        cbg_visits AS c
    INNER JOIN 
        cbgs AS g
    ON 
        c.census_block_group = g.censusblockgroup
);
ALTER TABLE cbgs_visits_long
RENAME COLUMN key TO home_cbg;
ALTER TABLE cbgs_visits_long
RENAME COLUMN value TO visitors_from_home_cbg;

-- Create index
CREATE INDEX cbgs_visits_long_dest_cbg_idx
ON cbgs_visits_long (destination_cbg);

-- Create index
CREATE INDEX cbgs_visits_long_home_cbg_idx
ON cbgs_visits_long (home_cbg);
"""

cbgs_with_geography = """
CREATE TEMPORARY TABLE t_cbgs_with_geo AS
    SELECT
        censusblockgroup,
        ST_Centroid(wkb_geometry::geography) AS geog
    FROM
        cbgs                                                                                                                                                                                                                                  
;
-- Create index
CREATE INDEX cbgs_cbg_idx
ON t_cbgs_with_geo (censusblockgroup);

-- Create index
CREATE INDEX cbgs_geo_idx
ON t_cbgs_with_geo 
USING GIST(geog);
"""

# CREATE A TABLE with distances between home CBG and destination CBG:
# shortest polygon-to point distance and centroid-to-point distance
cbgs_visits_breakdown_distances_table_statement = """
ALTER TABLE cbgs_visits_long
ADD COLUMN distance_centroids NUMERIC;

UPDATE cbgs_visits_long AS c
SET 
    distance_centroids = ST_Distance(h.geog, d.geog)
FROM 
    t_cbgs_with_geo AS h,
    t_cbgs_with_geo AS d
WHERE 
    c.destination_cbg = d.censusblockgroup AND
    c.home_cbg = h.censusblockgroup
;
"""

# Select all from the table
cbgs_visits_breakdown_distances_table_export_statement = """
SELECT 
    *
FROM
    cbgs_visits_long
;
"""

################################################################################

In [6]:
################## CBG-to-restaurants dataset creation #########################

# Build a table expanding visits-from-cbgs
print('Building visits breakdown.')
result = engine.execute(cbgs_visits_breakdown_table_statement)

# Build a table with cbg centroids
print('Building CBG centroids.')
result = engine.execute(cbgs_with_geography)

# Build a table with restaurant-cbg distances
print('Computing distnaces.')
result = engine.execute(cbgs_visits_breakdown_distances_table_statement)

# Get the expanded visits from CBGs (and corresponding distances) table
print('Exporting.')
cbgs_visits_breakdown = pd.read_sql(cbgs_visits_breakdown_distances_table_export_statement, 
                                           engine)

################################################################################

Building visits breakdown.
Building CBG centroids.
Computing distnaces.
Exporting.


In [7]:
########################### Close connection ###################################

engine.dispose()

################################################################################

In [8]:
cbgs_visits_breakdown

,destination_cbg,raw_visit_count,raw_visitor_count,home_cbg,visitors_from_home_cbg,distance_centroids
0,010010201001,54544,14460,010010208022,215,5730.711348
1,010010201001,54544,14460,010010205002,195,6464.368292
2,010010201001,54544,14460,010010208012,184,5120.143935
3,010010201001,54544,14460,010010208021,165,9982.780371
4,010010201001,54544,14460,010010206001,132,2452.117040
...,...,...,...,...,...,...
4131716,721537504004,24338,5811,721537503001,56,1860.262287
4131717,721537504005,12114,3244,721537506012,67,1973.738663
4131718,721537505022,14623,4294,721537506012,60,1467.869132
4131719,721537506011,6577,1966,721537506012,71,1355.331158


In [9]:
cbgs_visits_breakdown.dtypes

destination_cbg            object
raw_visit_count             int64
raw_visitor_count           int64
home_cbg                   object
visitors_from_home_cbg      int64
distance_centroids        float64
dtype: object

In [10]:
################## Get device and establishments data ##########################

# PostgreSQL connection
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
connection = engine.connect()

# Create a table with CBGs' device count and CBSA affiliation
cbg_table_statement = """
SELECT
    CONCAT(c.statefips, c.countyfips, c.tractcode) AS ct,
    c.censusblockgroup AS cbg,
    c.cbsa AS home_cbsa,
    h.number_devices_residing
FROM
    cbgs AS c
LEFT JOIN
    home AS h
ON
    c.censusblockgroup = h.census_block_group 
AND
    h.year = {year} 
AND
    h.month = {month}
;
""".format(year = year, month = month)

cbg_home = pd.read_sql(cbg_table_statement, engine)
# Remove duplicate CBGs
cbg_home = cbg_home.sort_values(['number_devices_residing'],
                                ascending = False).groupby('cbg').head(1)

# Create an establishments table with CBG affiliation and visits
visits_table_statement = """
SELECT 
    sname_place_id
FROM 
    visits
WHERE
    year = {year} AND
    month = {month}
;
""".format(year = year, month = month)
visits_month = pd.read_sql(visits_table_statement, engine)

establishments_table_statement = """
SELECT 
    sname_place_id,
    naics_first2,
    cbg
FROM 
    establishments
;
"""
establishments = pd.read_sql(establishments_table_statement, engine)

# Close the SQL connection-engine
engine.dispose()

################################################################################

In [11]:
############### Aggregate the establishments on the CBG level ##################

# Count open establishement by cbg
est_open_this_month = pd.merge(establishments,
                               visits_month,
                               how = 'inner',
                               validate = 'one_to_one')

del establishments, visits_month

cbg_establishments = est_open_this_month.groupby(['cbg',
                                                  'naics_first2']
                                                ).size().reset_index(name = 'cbg_naics')

del est_open_this_month

# Merge to get the CT of the CBG
cbg_establishments = pd.merge(cbg_establishments, 
                              cbg_home[['cbg','ct']], 
                              how = 'left',
                              on = 'cbg', 
                              validate = 'many_to_one')
# Count open establishments by ct
cbg_establishments['ct_naics'] = (cbg_establishments.groupby(['ct',
                                                              'naics_first2']
                                                            )['cbg_naics'].transform('sum') - 
                                  cbg_establishments['cbg_naics'])

cbg_establishments = cbg_establishments.pivot(index = 'cbg',
                                              columns = 'naics_first2',
                                              values = ['cbg_naics', 'ct_naics'])

################################################################################

In [12]:
cbg_establishments

cbg_naics                                                                                                            ct_naics                                                                                                           
naics_first2        11  21  22  23  31  32   33   42   44   45  48  49   51   52   53   54  55  56   61   62   71   72    81   92       11  21  22  23  31  32   33   42   44   45  48  49   51   52   53   54  55  56   61   62   71   72    81   92
cbg                                                                                                                                                                                                                                                  
010010201001       NaN NaN NaN NaN NaN NaN  NaN  NaN  2.0  NaN NaN NaN  NaN  1.0  NaN  NaN NaN NaN  NaN  1.0  NaN  NaN   1.0  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  0.0  NaN NaN NaN  NaN  0.0  NaN  NaN NaN NaN  NaN  0.0  NaN  NaN   0.0  NaN
010010202001       NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  2.0  NaN  1.0  NaN   1.0  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  6.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  3.0  NaN  2.0  NaN  11.0  NaN
010010202002       NaN NaN NaN NaN NaN NaN  NaN  NaN  6.0  2.0 NaN NaN  1.0  1.0  NaN  NaN NaN NaN  3.0  1.0  2.0  2.0  11.0  1.0      NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  0.0 NaN NaN  0.0  0.0  NaN  NaN NaN NaN  2.0  0.0  1.0  0.0   1.0  0.0
010010203001       NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   4.0  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  7.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   4.0  NaN
010010203002       NaN NaN NaN NaN NaN NaN  1.0  NaN  7.0  4.0 NaN NaN  NaN  NaN  1.0  1.0 NaN NaN  NaN  3.0  1.0  1.0   4.0  NaN      NaN NaN NaN NaN NaN NaN  0.0  NaN  1.0  0.0 NaN NaN  NaN  NaN  0.0  0.0 NaN NaN  NaN  0.0  0.0  0.0   4.0  NaN
...                ...  ..  ..  ..  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...   ...  ...      ...  ..  ..  ..  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...   ...  ...
721519510002       NaN NaN NaN NaN NaN NaN  NaN  1.0  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN      NaN NaN NaN NaN NaN NaN  NaN  0.0  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN
721537504004       NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  1.0  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  0.0  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN
721537504005       NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  0.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN
721537505011       NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  0.0  NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN
721537505022       NaN NaN NaN NaN NaN NaN  NaN  NaN  1.0  NaN NaN NaN  NaN  NaN  NaN  1.0 NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN      NaN NaN NaN NaN NaN NaN  NaN  NaN  0.0  NaN NaN NaN  NaN  NaN  NaN  0.0 NaN NaN  NaN  NaN  NaN  NaN   NaN  NaN

[200298 rows x 48 columns]

In [13]:
# Replace missing values with 0s
cbg_establishments.fillna(0, inplace = True)
# Flatten hierarchical index
cbg_establishments.set_axis([f'{x}{y}' for x, y in cbg_establishments.columns], 
                            axis = 1, inplace = True)
# Reset index
cbg_establishments.reset_index(inplace = True)
# Merge to get the CT of the destination CBG
cbg_establishments = pd.merge(cbg_establishments, 
                              cbg_home[['cbg','ct']], 
                              how = 'left',
                              on = 'cbg', 
                              validate = 'one_to_one')

In [14]:
# Rename columns
cbg_home.rename(columns = {'cbg': 'home_cbg', 'ct': 'home_ct'}, 
                inplace = True)
cbg_establishments.rename(columns = {'cbg': 'destination_cbg', 'ct': 'destination_ct'}, 
                          inplace = True)

In [15]:
cbg_home.head()

,home_ct,home_cbg,home_cbsa,number_devices_residing
202937,51013101401,510131014011,47900,34069.0
144825,36061003100,360610031001,35620,26437.0
140609,36005009300,360050093006,35620,14784.0
132231,32003006700,320030067001,29820,13154.0
46424,12095017103,120950171032,36740,12915.0


In [16]:
cbg_home.dtypes

home_ct                     object
home_cbg                    object
home_cbsa                   object
number_devices_residing    float64
dtype: object

In [17]:
cbg_establishments.head()

,destination_cbg,cbg_naics11,cbg_naics21,cbg_naics22,cbg_naics23,cbg_naics31,cbg_naics32,cbg_naics33,cbg_naics42,cbg_naics44,cbg_naics45,cbg_naics48,cbg_naics49,cbg_naics51,cbg_naics52,cbg_naics53,cbg_naics54,cbg_naics55,cbg_naics56,cbg_naics61,cbg_naics62,cbg_naics71,cbg_naics72,cbg_naics81,cbg_naics92,ct_naics11,ct_naics21,ct_naics22,ct_naics23,ct_naics31,ct_naics32,ct_naics33,ct_naics42,ct_naics44,ct_naics45,ct_naics48,ct_naics49,ct_naics51,ct_naics52,ct_naics53,ct_naics54,ct_naics55,ct_naics56,ct_naics61,ct_naics62,ct_naics71,ct_naics72,ct_naics81,ct_naics92,destination_ct
0,010010201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01001020100
1,010010202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,11.0,0.0,01001020200
2,010010202002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,01001020200
3,010010203001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,01001020300
4,010010203002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,01001020300


In [18]:
cbg_establishments.dtypes

destination_cbg     object
cbg_naics11        float64
cbg_naics21        float64
cbg_naics22        float64
cbg_naics23        float64
cbg_naics31        float64
cbg_naics32        float64
cbg_naics33        float64
cbg_naics42        float64
cbg_naics44        float64
cbg_naics45        float64
cbg_naics48        float64
cbg_naics49        float64
cbg_naics51        float64
cbg_naics52        float64
cbg_naics53        float64
cbg_naics54        float64
cbg_naics55        float64
cbg_naics56        float64
cbg_naics61        float64
cbg_naics62        float64
cbg_naics71        float64
cbg_naics72        float64
cbg_naics81        float64
cbg_naics92        float64
ct_naics11         float64
ct_naics21         float64
ct_naics22         float64
ct_naics23         float64
ct_naics31         float64
ct_naics32         float64
ct_naics33         float64
ct_naics42         float64
ct_naics44         float64
ct_naics45         float64
ct_naics48         float64
ct_naics49         float64
c

In [19]:
# Convert datatypes
cbg_home['number_devices_residing'] = cbg_home['number_devices_residing'].astype('Int64')
naics_col = [col for col in cbg_establishments if ('naics' in col)]
cbg_establishments[naics_col] = cbg_establishments[naics_col].astype('Int64')

In [20]:
# Merge the datasets
cbgs_visits_breakdown = pd.merge(cbgs_visits_breakdown,
                                 cbg_home,
                                 how = 'left',
                                 on = 'home_cbg',
                                 validate = 'many_to_one')

cbgs_visits_breakdown = pd.merge(cbgs_visits_breakdown,
                                 cbg_establishments[['destination_cbg', 'destination_ct']],
                                 how = 'left',
                                 on = 'destination_cbg',
                                 validate = 'many_to_one')

In [21]:
# Selecting one destination CBG within a CT
cbgs_visits_breakdown = cbgs_visits_breakdown.sort_values(['visitors_from_home_cbg'],
                                                          ascending = False
                                                         ).groupby(['home_cbg',
                                                                    'destination_ct']).head(1)

In [22]:
#cbgs_visits_breakdown = pd.merge(cbgs_visits_breakdown,
#                                 cbg_establishments[['destination_cbg'] + naics_col],
#                                 how = 'left',
#                                 on = 'destination_cbg',
#                                 validate = 'many_to_one')

In [23]:
cbgs_visits_breakdown.head()

,destination_cbg,raw_visit_count,raw_visitor_count,home_cbg,visitors_from_home_cbg,distance_centroids,home_ct,home_cbsa,number_devices_residing,destination_ct
1967311,250173530003,722808,410077,250173531011,13323,4.572688e+02,25017353101,14460,10637,25017353000
719476,090010430003,232818,50788,250173531011,11786,2.384135e+05,25017353101,14460,10637,09001043000
4036293,530330067001,118669,48849,250173531011,9104,4.007908e+06,25017353101,14460,10637,53033006700
3504893,481576731021,546472,58573,481576731011,4006,4.145194e+03,48157673101,26420,7286,48157673102
3609726,482014553001,547959,91272,481576731011,3944,5.416379e+03,48157673101,26420,7286,48201455300


In [24]:
cbgs_visits_breakdown.dtypes

destination_cbg             object
raw_visit_count              int64
raw_visitor_count            int64
home_cbg                    object
visitors_from_home_cbg       int64
distance_centroids         float64
home_ct                     object
home_cbsa                   object
number_devices_residing      Int64
destination_ct              object
dtype: object

In [25]:
cbgs_visits_breakdown.shape

(2496892, 10)

In [26]:
cbgs_visits_breakdown.sort_values(by = 'home_cbg', inplace = True)

In [27]:
# Construct the within-cbg destination-cbg choice counts
cbgs_visits_breakdown['visits_from_home_cbg'] = (cbgs_visits_breakdown['visitors_from_home_cbg'] *
                                                 cbgs_visits_breakdown['raw_visit_count'] /
                                                 cbgs_visits_breakdown['raw_visitor_count'])

In [28]:
# Construct the outside-good choice count
#cbgs_visits_breakdown['outside_good_count'] = (
#    cbgs_visits_breakdown[
#        'visits_from_home_cbg'
#    ].groupby(cbgs_visits_breakdown['home_cbg']
#             ).transform('sum')
#)
# Total choices made 31 * restaurants_visits_breakdown['number_devices_residing']:
#cbgs_visits_breakdown['outside_good_count'] = (
#    31 * cbgs_visits_breakdown['number_devices_residing'] - 
#    cbgs_visits_breakdown['outside_good_count']
#)

In [29]:
# Treat home ct as the outside good
home_ct_rows = cbgs_visits_breakdown['home_ct'] == cbgs_visits_breakdown['destination_ct']
away_ct_rows = cbgs_visits_breakdown['home_ct'] != cbgs_visits_breakdown['destination_ct']

outside_good = cbgs_visits_breakdown[['home_cbg', 'visits_from_home_cbg']].loc[home_ct_rows].copy()
outside_good.rename(columns = {'visits_from_home_cbg': 'outside_good'}, inplace = True)

# Only away-ct treated as destination choices
cbgs_visits_breakdown = cbgs_visits_breakdown.loc[away_ct_rows]
cbgs_visits_breakdown.drop('destination_ct', axis = 1, inplace = True)

# Merge to get establishments count in cbg and rest of ct
cbgs_visits_breakdown = pd.merge(cbgs_visits_breakdown,
                                 cbg_establishments, 
                                 how = 'left', 
                                 on = 'destination_cbg',
                                 validate = 'many_to_one')

# Merge to get the count of the outside good choices
cbgs_visits_breakdown = pd.merge(cbgs_visits_breakdown,
                                 outside_good, 
                                 how = 'left', 
                                 on = 'home_cbg',
                                 validate = 'many_to_one')

In [30]:
cbgs_visits_breakdown.head()

,destination_cbg,raw_visit_count,raw_visitor_count,home_cbg,visitors_from_home_cbg,distance_centroids,home_ct,home_cbsa,number_devices_residing,visits_from_home_cbg,cbg_naics11,cbg_naics21,cbg_naics22,cbg_naics23,cbg_naics31,cbg_naics32,cbg_naics33,cbg_naics42,cbg_naics44,cbg_naics45,cbg_naics48,cbg_naics49,cbg_naics51,cbg_naics52,cbg_naics53,...,ct_naics21,ct_naics22,ct_naics23,ct_naics31,ct_naics32,ct_naics33,ct_naics42,ct_naics44,ct_naics45,ct_naics48,ct_naics49,ct_naics51,ct_naics52,ct_naics53,ct_naics54,ct_naics55,ct_naics56,ct_naics61,ct_naics62,ct_naics71,ct_naics72,ct_naics81,ct_naics92,destination_ct,outside_good
0,010010202002,61089,7123,010010201002,94,2896.601939,01001020100,33860,149,806.172399,0,0,0,0,0,0,0,0,6,2,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,01001020200,343.257538
1,010010203002,91053,20907,010010201002,96,3721.388405,01001020100,33860,149,418.093844,0,0,0,0,0,0,1,0,7,4,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,01001020300,343.257538
2,011010033021,235672,52051,010010201002,55,28737.260769,01001020100,33860,149,249.024226,0,0,0,0,0,0,1,0,9,7,0,2,2,4,0,...,0,0,0,0,0,0,0,6,3,0,0,0,16,0,0,0,1,2,37,0,0,2,0,01101003302,343.257538
3,010010205002,156021,21781,010010201002,95,6429.198098,01001020100,33860,149,680.501125,0,0,0,0,0,1,0,0,13,10,0,2,0,14,1,...,0,0,0,0,0,0,0,5,4,0,0,0,2,0,1,0,0,0,3,1,2,7,0,01001020500,343.257538
4,010010204003,35914,6349,010010201002,83,4996.614093,01001020100,33860,149,469.501024,0,0,0,0,0,0,0,0,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0,0,1,0,0,5,0,01001020400,343.257538


In [31]:
################### Export data to process on local computer ###################

# Export path
cbgs_logit_file_path = os.path.join(output_folder_path,
                               'data_cbgs_logit.csv')
# Perform export
cbgs_visits_breakdown.to_csv(cbgs_logit_file_path,
                             index = False)

################################################################################